# Imports

In [1]:
import argparse

import sys
sys.path.append('/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/scripts/')

from settings import LABELS_GENERATOR_DEFAULT_SETTINGS
from labels_generator import LabelsGenerator

In [2]:
default_params_dict = LABELS_GENERATOR_DEFAULT_SETTINGS
default_params_dict['dev_dataset_dir'] = '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev'
default_params_dict['clients_lines_max'] = 10
default_params_dict['impostors_lines_max'] = 15
default_params = argparse.Namespace(**default_params_dict)

labels_generator = LabelsGenerator(default_params)

In [3]:
labels_generator.dev_speakers_dict = labels_generator.generate_speakers_dict(
            load_path = labels_generator.params.dev_dataset_dir,
        )

Loading dev data...
Dev data loaded.


/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/scripts/labels_generator.py:57: UserWarning: Ambiguous directory path: /home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09210/ido93Up7-ig
  warnings.warn(f"Ambiguous directory path: {dir_path}")


In [4]:
labels_generator.dev_speakers_dict.keys()

dict_keys(['id09092', 'id09163', 'id09217', 'id09229', 'id09087', 'id09116', 'id09236', 'id09198', 'id09188', 'id09171', 'id09147', 'id09088', 'id09213', 'id09161', 'id09262', 'id09192', 'id09207', 'id09123', 'id09172', 'id09222', 'id09152', 'id09099', 'id09139', 'id09211', 'id09175', 'id09234', 'id09143', 'id09108', 'id09121', 'id09095', 'id09252', 'id09224', 'id09242', 'id09197', 'id09200', 'id09145', 'id09119', 'id09091', 'id09216', 'id09109', 'id09094', 'id09204', 'id09201', 'id09098', 'id09125', 'id09138', 'id09240', 'id09110', 'id09112', 'id09186', 'id09202', 'id09227', 'id09225', 'id09179', 'id09190', 'id09237', 'id09154', 'id09111', 'id09185', 'id09162', 'id09100', 'id09214', 'id09245', 'id09235', 'id09146', 'id09124', 'id09255', 'id09218', 'id09155', 'id09164', 'id09215', 'id09231', 'id09263', 'id09212', 'id09156', 'id09153', 'id09160', 'id09193', 'id09144', 'id09243', 'id09238', 'id09083', 'id09271', 'id09230', 'id09194', 'id09268', 'id09159', 'id09232', 'id09199', 'id09189',

In [5]:
def train_valid_split_dict(speakers_dict, train_speakers_pctg, random_split = True):

        print(f"Spliting data into train and valid...")
        
        # We are going to split speakers_dict in train and valid using speaker_id's
        # ie, all audios of the same speaker goes to same split.
        
        speakers_set = set(speakers_dict.keys())
        num_speakers = len(speakers_set)
        # How many speakers will contain the train split
        train_speakers_final_index = int(num_speakers * train_speakers_pctg)

        # The split can be done using speakers' ids order or randomly
        
        # Order the speakers
        speakers_list = list(speakers_set)
        if random_split == False:
            speakers_list.sort()
        elif random_split == True:
            random.shuffle(speakers_list)

        # Add 0 to total_speakers-1 labels
        for i, speaker in enumerate(speakers_list):
            speakers_dict[speaker]["speaker_num"] = i

        # Sort in order of labels for better understanding
        speakers_dict = {k: v for k, v in sorted(speakers_dict.items(), key=lambda item: item[1]["speaker_num"])}
            
        train_speakers_dict = speakers_dict.copy()
        valid_speakers_dict = speakers_dict.copy()

        for speaker in speakers_dict.keys():

            speaker_num = speakers_dict[speaker]["speaker_num"]

            if speaker_num >= train_speakers_final_index:
                del train_speakers_dict[speaker]
            else:
                del valid_speakers_dict[speaker]

        train_speakers_num = len(train_speakers_dict.keys())
        valid_speakers_num = len(valid_speakers_dict.keys())
        total_speakers_num = train_speakers_num + valid_speakers_num
        if total_speakers_num != num_speakers:
            raise Exception("total_speakers_num does not match total_speakers_num!")
        train_speakers_pctg = train_speakers_num * 100 / total_speakers_num
        valid_speakers_pctg = valid_speakers_num * 100 / total_speakers_num
        
        train_files_num = len(list(itertools.chain.from_iterable([value["files_paths"] for value in train_speakers_dict.values()])))
        valid_files_num = len(list(itertools.chain.from_iterable([value["files_paths"] for value in valid_speakers_dict.values()])))
        total_files_num = train_files_num + valid_files_num
        train_files_pctg = train_files_num * 100 / total_files_num
        valid_files_pctg = valid_files_num * 100 / total_files_num
        
        
        print(f"{train_speakers_num} speakers ({train_speakers_pctg:.1f}%) with a total of {train_files_num} files ({train_files_pctg:.1f}%) in training split.")
        print(f"{valid_speakers_num} speakers ({valid_speakers_pctg:.1f}%) with a total of {valid_files_num} files ({valid_files_pctg:.1f}%) in validation split.")
        
        print(f"Data splited.")
        
        return train_speakers_dict, valid_speakers_dict

In [11]:
a = train_valid_split_dict(
    labels_generator.dev_speakers_dict, 
    labels_generator.params.train_speakers_pctg, 
    labels_generator.params.random_split,)

Spliting data into train and valid...
70 speakers (69.3%) with a total of 11136 files (69.4%) in training split.
31 speakers (30.7%) with a total of 4902 files (30.6%) in validation split.
Data splited.


In [7]:
import random
import itertools

In [12]:
a

({'id09190': {'files_paths': {'/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/1IUmZ7oQRUM/00001',
    '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/1IUmZ7oQRUM/00002',
    '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/1IUmZ7oQRUM/00003',
    '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/1IUmZ7oQRUM/00004',
    '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/6myuG2_d68s/00005',
    '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/6myuG2_d68s/00006',
    '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/6myuG2_d68s/00007',
    '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/6myuG2_d68s/00008',
    '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/6myuG2_d68s/00009',
    '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/6myuG2_d68s/00010',
    '/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev/id09190/6myuG2_d68s/000

In [ ]:
! python /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/scripts/labels_generator.py -h

# 1 - Test the original train class

In [ ]:
! python /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/scripts/train.py -h

In [ ]:
! python /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/scripts/train.py \
--train_data_dir 